<a href="https://colab.research.google.com/github/vegarab/msc-qg/blob/master/notebooks/NonInt_XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning XLNet For Generation Tasks
https://nonint.com/2020/03/27/fine-tuning-xlnet-for-generation-tasks/


In [1]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 12.2MB/s 
     |████████████████████████████████| 1.1MB 35.4MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5ec138d0029a0aebfeba125494e4951380e7576d24bcba66162759f32ef81a69
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [32]:
import torch
import transformers

import json

In [8]:
!curl -LO https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.1M  100 40.1M    0     0  22.1M      0  0:00:01  0:00:01 --:--:-- 22.1M


In [10]:
with open('train-v2.0.json') as train_file:
    train_data_raw = json.load(train_file)

In [29]:
#print(len(train_data_raw['data']))
sample = train_data_raw['data'][0]['paragraphs'][0]

context = sample['context']
question = sample['qas'][0]['question']
answer = sample['qas'][0]['answers'][0]['text']

sample_data = {'context': context,
               'question': question,
               'answer': answer}

print(sample_data)

train = [sample_data]
torch.save(train, 'train.pt')

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answer': 'in the late 1990s'}


In [33]:
tokenizer = transformers.XLNetTokenizer.from_pretrained('xlnet-base-cased')
MAX_SEQ_LEN = 700
PREDICT_LEN = 48

bos_token_tensor = torch.tensor([tokenizer.bos_token_id], dtype=torch.long)
eos_token_tensor = torch.tensor([tokenizer.eos_token_id], dtype=torch.long)
sep_token_tensor = torch.tensor([tokenizer.sep_token_id], dtype=torch.long)

target = torch.cat([sample_data['question'], eos_token_tensor], dtype=torch.long)
target_len = target.shape[0]

# Truncate off the end of the target if it is too long
if target_len > PREDICT_LEN:
    target = target[: PREDICT_LEN]
    # Keep the <eos> token post-truncation
    target[-1] = tokenizer.eos_token_id

# The target will be end of the sequence, so append that token onto it.
# Also pad target to max length. This is done to force the model to predict the actual end of sequence
pads_needed = PREDICT_LEN - target_len
if pads_needed > 0:
    target_padding = torch.full((pads_needed,), tokenizer.pad_token_id, dtype=torch.long)
    target = torch.cat([target, target_padding])
target_len = PREDICT_LEN

text = torch.cat([bos_token_tenor, text, sep_token_tensor, target])

# Create attention mask is all 0s for the text; all tokens attend to each other. For the target, our goal is
# sequential generation, so the permutation will also be sequential. We won't let the text perform any
# attention on the title


TypeError: ignored

In [34]:
tokenizer.tokenize(context, answer)

TypeError: ignored

In [35]:
print(context)

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
